<h1 style='text-align: center; text-decoration: underline;'>argentum</h1>
<p style='text-align: center;'>
    Macroeconomic Modeling of Currency Debasement in the Roman Empire
</p>

## Import Libraries

In [1]:
import pandas as pd
import numpy as np

## Data Construction

### Archaeology Data Service Analysis of Roman Silver coins

In [51]:
# Import data from Archaeology Data Service Analysis of Roman Silver coins 
# Link: https://archaeologydataservice.ac.uk/archives/view/coins_lt_2005/index.cfm
ads_roman_df_temp = pd.read_excel('datasets/Denarii_JC_to_Trajans_reform_final.xlsx')
ads_roman_df = ads_roman_df_temp.drop(columns=['DENOMINATION', 
                                                    'DONOR',
                                                    'DONORREF',
                                                    'WALKER',
                                                    'MINT',
                                                    'OBVERSE',
                                                    'REVERSE',
                                                    'HOARD',
                                                    'WEIGHT',
                                                    'COMMENTS',
                                                    'BULLION',
                                                    'Unnamed: 28', 
                                                    'Unnamed: 29', 
                                                    'Unnamed: 30', 
                                                    'Unnamed: 31', 
                                                    'Unnamed: 32'])

### Data Cleaning
Some of the data have negative silver percentages. This happens in ICP-AES (the spectrometry method used) when a metal is so scarce in the object that the sensor reads "below" its zero-calibration point. This effectively means that there is 0% Silver in this coin.

We are only interested in coins that are a <i>'Denarius'</i> (Silver Coin). Although all coins in the dataset are classified as <i>'Denarius'</i> in 'DENOMINATION', we have some data where Copper is the primary metal in the coin.

Coins that have over 90% of Copper are likely <i>'Sestertius'</i> (Bronze/Copper small change).

If we include this row in your regression model to predict "Silver Debasement," this coin will look like a massive economic crash because the silver drops to zero. But it's just a penny, not a dollar. 

So
<ul>
    <li> We will keep coins that have more than 20% of their composition as Silver
    <li> We will filter out coins that have more than 75% of their composition as Copper
</ul>

In [55]:
# Data Cleaning
ads_silver_coins = ads_roman_df[ads_roman_df['SILVER'] > 20]
ads_only_silver_coins = ads_silver_coins[ads_silver_coins['COPPER'] < 90]

# Format of REF | Emperor | Silver Content
ads_only_silver_coins = ads_only_silver_coins[['REF', 'EMPEROR', 'SILVER']]


In [58]:
print(ads_only_silver_coins.sort_values(by='SILVER', ascending=False))

        REF    EMPEROR      SILVER
722   WM190   Tiberius  100.000566
718   WM152   Tiberius   99.982151
255  KB1311      Galba   99.809402
254  KB1310      Galba   99.745357
690      w4   Caligula   99.585976
..      ...        ...         ...
299    LW36      Titus   70.224958
552  SHK172      Titus   70.024435
396      M6      Galba   56.862404
399     M62  Vespasian   49.175715
127   BM036  Vespasian   44.230033

[723 rows x 3 columns]


## Linear Regression Analysis